# Medical Data Visualizer

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Import the data from medical_examination.csv and assign it to the df variable.

In [5]:
df = pd.read_csv('medical_examination.csv')

df.head(3)

,Feature,Variable Type,Variable,Value Type
0,Age,Objective Feature,age,int (days)
1,Height,Objective Feature,height,int (cm)
2,Weight,Objective Feature,weight,float (kg)


## Add an overweight column to the data. To determine if a person is overweight, first calculate their BMI by dividing their weight in kilograms by the square of their height in meters. If that value is > 25 then the person is overweight. Use the value 0 for NOT overweight and the value 1 for overweight.


In [6]:
df = df['overweight'] = (df['weight'] / ((df['height'] / 100) ** 2)).apply(lambda x: 1 if x > 25 else 0)

KeyError: 'weight'


Normalize data by making 0 always good and 1 always bad. If the value of cholesterol or gluc is 1, set the value to 0. If the value is more than 1, set the value to 1.
Draw the Categorical Plot in the draw_cat_plot function.
Create a DataFrame for the cat plot using pd.melt with values from cholesterol, gluc, smoke, alco, active, and overweight in the df_cat variable.
Group and reformat the data in df_cat to split it by cardio. Show the counts of each feature. You will have to rename one of the columns for the catplot to work correctly.
Convert the data into long format and create a chart that shows the value counts of the categorical features using the following method provided by the seaborn library import: sns.catplot().
Get the figure for the output and store it in the fig variable.
Do not modify the next two lines.
Draw the Heat Map in the draw_heat_map function.
Clean the data in the df_heat variable by filtering out the following patient segments that represent incorrect data:
diastolic pressure is higher than systolic (Keep the correct data with (df['ap_lo'] <= df['ap_hi']))
height is less than the 2.5th percentile (Keep the correct data with (df['height'] >= df['height'].quantile(0.025)))
height is more than the 97.5th percentile
weight is less than the 2.5th percentile
weight is more than the 97.5th percentile
Calculate the correlation matrix and store it in the corr variable.
Generate a mask for the upper triangle and store it in the mask variable.
Set up the matplotlib figure.
Plot the correlation matrix using the method provided by the seaborn library import: sns.heatmap().
Do not modify the next two lines.